<a href="https://colab.research.google.com/github/honeyd3wy/PyTorch/blob/master/n122_chi_square_test_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://i.imgur.com/RDAD11M.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 1 / SPRINT 2 / NOTE 2*

# 📝 Assignment

# 카이제곱검정


## 1. 공공데이터 조작 연습. 

국가 통계 포털에서 아래 이미지를 참조하여

<https://kosis.kr/statisticsList/statisticsListIndex.do?menuId=M_01_01&vwcd=MT_ZTITLE&parmTabId=M_01_01#SelectStatsBoxDiv>

**2020년 8월**에 해당하는 규모별 미분양현황 에 대한 데이터셋을 생성하세요. 

<img src='https://i.imgur.com/aZc4UJO.png' width = 500>

- `60m이하`, `60~85m`, `85m초과`의 3개 규모와 (column)
- `서울`, `대전`, `대구`, `부산` 4개의 지역을 포함해야합니다. (row)
- `민간부문`만 포함합니다
- 데이터가 없는 경우는 0으로 처리하세요.

이후 데이터셋을 colab으로 불러오세요. 이때 변수의 이름은 `df`를 사용합니다.


In [1]:
col = ['~60m2', '60~85m2', '85m2~']
row = ['서울', '대전', '대구', '부산']

In [2]:
import pandas as pd

parcel = pd.read_csv('규모별_미분양현황_2020_08.csv', encoding='CP949')
# 유니코드 디코드 에러, UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc1 in position 26: invalid start byte
# 해결법: encoding='CP949' or encoding='latin'
# https://rfriend.tistory.com/250

parcel.columns = ['시도', '부문', '규모', 'value']
# 열 이름 전체 변경: .columns = ['c1', 'c2', ...]
# 일부 변경: .rename(columns={'before':'after'})
parcel

,시도,부문,규모,value
0,서울,민간부문,60㎡이하,54
1,서울,민간부문,60∼85㎡,2
2,서울,민간부문,85㎡초과,0
3,부산,민간부문,60㎡이하,577
4,부산,민간부문,60∼85㎡,735
5,부산,민간부문,85㎡초과,142
6,대구,민간부문,60㎡이하,143
7,대구,민간부문,60∼85㎡,1437
8,대구,민간부문,85㎡초과,44
9,대전,민간부문,60㎡이하,782


In [3]:
#  문제 조건에 맞도록 출력하기 위해 tidy -> wide
df = parcel.pivot_table(index = '시도', columns = '규모', values = 'value')
df

규모,60∼85㎡,60㎡이하,85㎡초과
시도,,,
대구,1437,143,44
대전,1,782,0
부산,735,577,142
서울,2,54,0


## 2. 지역에 대해서 one-sample chi-square test를 실행, 해당 결과를 `chi1`에 저장 후 설명해보세요.

예시) 만약 **9월달 데이터**를 기준으로 한다면
```python
 [52+2+0, 590+665+142, 113+1061+42, 772+1+0]
```
을 비교 하게 될 것입니다.




In [4]:
df_obs = pd.DataFrame(df.sum(axis=1))
#df_obs = df_obs.reset_index()
df_obs.columns = ['sum_2020_08']
df_obs

,sum_2020_08
시도,
대구,1624
대전,783
부산,1454
서울,56


In [5]:
# One sample chi-square(적합도 검정)

# 가설 설정 : 미분양수가 지역별로 차이가 있다고 할 수 있는가?
#   H0 : 서울.sum == 대구.sum == 대전.sum == 부산.sum (차이가 없다.)
#   H1 : not H0 (차이가 있다.)

import scipy.stats as stats
chi1 = stats.chisquare(df_obs['sum_2020_08'], axis=None)
chi1

Power_divergenceResult(statistic=1564.4572376818994, pvalue=0.0)

 $\chi^2$ =1564.457, p-value=0.0

p-value 가 매우 작으므로 귀무가설을 기각한다.

따라서 **미분양수가 지역별로 차이가 있다**고 할 수 있다.

In [6]:
# stats.chi2 : 입력한 >>검정통계량<<의 값에 따라 카이제곱분포식에서 값을 가져오는 메서드
#   cdf(누적분포함수), 즉 확률값 반환
#   pdf(확률밀도함수)

#from scipy import stats
#1- stats.chi2.cdf(1564.4572376818994, df=3)

## 3. 지역과 규모에 대해서 two-sample chi-square test를 실행, 해당 결과를 `chi2`에 저장 후 설명해보세요.

예시) **9월달 데이터**를 기준으로 한다면

| |-60 | 60-85 | 85- |
|:-:|:-:|:-:|:-:|
|서울|52|2|0|
|대전|772|1|0|
|대구|113|1061|42|
|부산|590|665|142|

에 대해서 검정해야 할 겁니다.




In [7]:
df

규모,60∼85㎡,60㎡이하,85㎡초과
시도,,,
대구,1437,143,44
대전,1,782,0
부산,735,577,142
서울,2,54,0


In [8]:
# Two sample chi-sqare(독립성 검정)
# 가설 설정: 지역과 규모간에 관련성이 있다고 할 수 있는가?
#   H0 : 관련성이 없다.(독립적이다)
#   H1 : 관련성이 있다.(독립적이지 않다)

from scipy import stats
chi2 = stats.chi2_contingency(df)
chi2

# 이렇게 하면 안 된다... chi2 메서드에는 검정통계량 값을 넣어야 함.
# stats.chi2.cdf(df,df=6)

(2064.5767314171994, 0.0, 6, array([[901.76155221, 645.12228746,  77.11616033],
        [434.77789124, 311.04110288,  37.18100587],
        [807.36533061, 577.59101353,  69.04365586],
        [ 31.09522594,  22.24559612,   2.65917794]]))

p-value가 아주 작으므로 귀무가설을 기각한다.

따라서, **지역과 규모간에 관련성이 있다**고 할 수 있다.

## 4. 2번에 대해서 NumPy 를 사용하여 (Scipy를 사용하지 않고) $\chi^2$ test 시행 후 2번의 결과와 비교해보세요. 

- `obs`, `exp`, `chi`라는 변수를 사용해야합니다.


In [9]:
df_obs

,sum_2020_08
시도,
대구,1624
대전,783
부산,1454
서울,56


In [10]:
import numpy as np
obs = np.array(df_obs['sum_2020_08'])
obs

array([1624,  783, 1454,   56])

In [11]:
total_obs = np.sum(obs)
total_obs

3917

In [12]:
exp = np.array([total_obs,total_obs,total_obs,total_obs])
exp = exp/4
exp

array([979.25, 979.25, 979.25, 979.25])

In [13]:
squared = np.power(obs - exp, 2)
squared

array([415702.5625,  38514.0625, 225387.5625, 852390.5625])

In [14]:
chi = np.sum(squared / exp)
chi

1564.4572376818994

2번 문제의 chi1의 결과는 검정통계량 $\chi^2$=1564.4572376818994로, 새로 계산한 chi와 같다.


## 🔥 도전 과제

아래 세가지를 다하면 됩니다.

### 1. Function

4번에서 사용한 one sample chisquare test를 함수의 형태로 변경하세요.

In [15]:
v1 = [18,22,20,15,23,22]
v2 = [5,23,26,19,24,23]

def myChisq(value):
    obs = np.array(value)
    total_obs = np.sum(obs)
    exp = np.array([])
    for i in range(len(value)):
        exp = np.append(exp, total_obs)
    exp = exp/len(value)
    squared = np.power(obs - exp, 2)
    chi = np.sum(squared / exp)
    return chi, 1- stats.chi2.cdf(chi, df=len(value)-1)

# chisquare값과 pvalue를 출력해야합니다.
myChisq(v1)

(2.3000000000000003, 0.8062668698851285)

In [16]:
# test v1
stats.chisquare(v1, axis=None)

Power_divergenceResult(statistic=2.3000000000000003, pvalue=0.8062668698851285)

In [17]:
myChisq(v2)

(14.8, 0.011251979028327308)

In [18]:
# test v2
stats.chisquare(v2, axis=None)

Power_divergenceResult(statistic=14.8, pvalue=0.011251979028327346)

### 2. ANOVA

아래 링크를 참조하여 ANOVA 에 대한 글을 읽고

<https://partrita.github.io/posts/ANOVA-python/>

다음 `4개 그룹에 대해서 평균의 차이가 있는지`에 대한 가설 검정을 시행하세요.

A : `38 33 35 92 76 97 88 41 11  9`

B : `18 52 62 48 30 40 87 12 97 82`

C :  `28 90  5 49 66 73 96 80  4 17`

D : ` 8 99  4 12  7 64 18 10  9 20`



In [19]:
# 데이터셋 불러오기
df_anova = pd.DataFrame({'A': [38, 33, 35, 92, 76, 97, 88, 41, 11, 9],
                     'B': [18, 52, 62, 48, 30, 40, 87, 12, 97, 82],
                     'C': [28, 90, 5, 49, 66, 73, 96, 80, 4, 17],
                     'D': [8, 99, 4, 12, 7, 64, 18, 10, 9, 20]})
df_anova

,A,B,C,D
0,38,18,28,8
1,33,52,90,99
2,35,62,5,4
3,92,48,49,12
4,76,30,66,7
5,97,40,73,64
6,88,87,96,18
7,41,12,80,10
8,11,97,4,9
9,9,82,17,20


In [20]:
# 가설 설정
#   H0 : A.mean == B.mean == C.mean == D.mean
#   H1 : not H0

fvalue, pvalue = stats.f_oneway(df_anova['A'],df_anova['B'],df_anova['C'],df_anova['D'])
print(fvalue, pvalue)

1.7249594239128412 0.17920877113948797


In [21]:
# ANOVA table 만들어보기

# tidy화
df_anova_melt = pd.melt(df_anova.reset_index(), id_vars=['index'], value_vars=['A', 'B', 'C', 'D'])
df_anova_melt.columns = ['index', 'treatments', 'value']
df_anova_melt

,index,treatments,value
0,0,A,38
1,1,A,33
2,2,A,35
3,3,A,92
4,4,A,76
5,5,A,97
6,6,A,88
7,7,A,41
8,8,A,11
9,9,A,9


In [22]:
# 패키지 설치
# install
#! pip install bioinfokit

# upgrade to latest version
#! pip install bioinfokit --upgrade

In [23]:
# 경고 메세지 무시하기
import warnings
warnings.filterwarnings('ignore')

from bioinfokit.analys import stat

res = stat()
res.anova_stat(df=df_anova_melt, res_var='value', anova_model='value ~ C(treatments)')
res.anova_summary

,df,sum_sq,mean_sq,F,PR(>F)
C(treatments),3.0,5393.675,1797.891667,1.724959,0.179209
Residual,36.0,37522.100,1042.280556,NaN,NaN


p-value > 0.1이므로 귀무가설을 채택한다.

즉, 그룹 A, B, C, D의 평균은 **하나라도 다르다고 보기 어렵다**.
